### Importing Libraries

In [1]:
import cv2 as cv
import time 
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
import math
import osascript

objc[66356]: Class CaptureDelegate is implemented in both /Users/adilbhatti/anaconda3/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x1516124d0) and /Users/adilbhatti/anaconda3/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x117e34860). One of the two will be used. Which one is undefined.
objc[66356]: Class CVWindow is implemented in both /Users/adilbhatti/anaconda3/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x151612520) and /Users/adilbhatti/anaconda3/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1125d8a68). One of the two will be used. Which one is undefined.
objc[66356]: Class CVView is implemented in both /Users/adilbhatti/anaconda3/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x151612548) and /Users/adilbhatti/anaconda3/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x1125d8a90). One of the two will be used. Which one is undefined.
objc[66356]: Class CVSlider is implemented in both /U

In [22]:
!rm -r mediapipe



rm: mediapipe: No such file or directory


In [23]:
!pip


Usage:   
  pip <command> [options]

Commands:
  install                     Install packages.
  download                    Download packages.
  uninstall                   Uninstall packages.
  freeze                      Output installed packages in requirements format.
  list                        List installed packages.
  show                        Show information about installed packages.
  check                       Verify installed packages have compatible dependencies.
  config                      Manage local and global configuration.
  search                      Search PyPI for packages.
  cache                       Inspect and manage pip's wheel cache.
  index                       Inspect information available from package indexes.
  wheel                       Build wheels from your requirements.
  hash                        Compute hashes of package archives.
  completion                  A helper command used for command completion.
  debug                    

### Hand Landmark Detection Model

In [2]:
# hands class
mp_hands = mp.solutions.hands

# setting up hand functions
hands = mp_hands.Hands(min_detection_confidence = 0.70)

# drawing class
mp_draw = mp.solutions.drawing_utils

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


### Making a function to preform hand landmark detection

In [3]:
# Returns resulting image and results
def detectHandLandmarks(img, hands, draw=True):
    output_img = img.copy()
    imgRGB = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    
    if results.multi_hand_landmarks and draw:
        for hand_landmark in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(image = output_img, landmark_list = hand_landmark,
                                      connections = mp_hands.HAND_CONNECTIONS,
                                      landmark_drawing_spec = mp_draw.DrawingSpec(color=(0,0,255),
                                                                                  thickness=5, circle_radius=5),
                                      connection_drawing_spec=mp_draw.DrawingSpec(color = (255,255,255),
                                                                                     thickness = 2, circle_radius = 10))
    return (output_img, results)      

### Miscilaneous

In [6]:
# Text
font                   = cv.FONT_HERSHEY_SIMPLEX
fontScale              = 2
fontColor              = (255,255,0)
thickness              = 5
lineType               = 2


### Video Capture

In [11]:
cap = cv.VideoCapture(0)
ptime = ctime = 0

while cap.isOpened():
    ret, img = cap.read()
    if not ctime:
        h, w, c = img.shape
    if ret:
        drawn_img, results = detectHandLandmarks(img, hands)
        ctime = time.time()
        fps = 1/(ctime-ptime)
        ptime = ctime
        cv.putText(drawn_img,f'FPS: {int(fps)}', (50, 70), font, fontScale,fontColor,thickness,lineType)
        thumbs_up = True
        # Labeling the landmarks also calculates if thumbs is highest y coordinate
        if results.multi_hand_landmarks:
            for handLms in results.multi_hand_landmarks:
                thumb_y = (handLms.landmark[4].y * h)
                for i, lm in enumerate(handLms.landmark):
                        cx = math.ceil(w * lm.x)
                        cy = math.ceil(h * lm.y)
                        thumbs_up &= (thumb_y < cy) 
                        if not i:
                            cv.circle(drawn_img, (cx, cy), 30, (0, 0, 0), -1)
                            cv.putText(drawn_img,f'{i}', (cx-10, cy+5), font, 1,(255,255,255),thickness,lineType)
                        elif not i%4 :
                            cv.circle(drawn_img, (cx, cy), 30, (0, 0, 255), -1)
                            cv.putText(drawn_img,f'{i}', (cx-10, cy+5), font, 1,(255,255,255),thickness,lineType)
                        elif not (i+2)%4 :
                            cv.circle(drawn_img, (cx, cy), 30, (255, 0, 0), -1)
                            cv.putText(drawn_img,f'{i}', (cx-10, cy+5), font, 1,(255,255,255),thickness,lineType)
        else:
            thumbs_up = False
        
        if thumbs_up:
            cv.putText(drawn_img,"Thumbs UP!", (w - 300, 70), font, 1,(205,20,205),thickness,lineType)
        cv.imshow("Webcam", drawn_img)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv.destroyAllWindows()

### Building finger counter (Lets build based on distance from wrist for the tip of finger)

In [10]:
def fingerCounter(img, hands, draw = True):
    finger_status = {
    "RIGHT THUMB": False,
    "RIGHT INDEX": False,
    "RIGHT MIDDLE": False,
    "RIGHT RING": False,
    "RIGHT PINKY": False,
    "LEFT THUMB": False,
    "LEFT INDEX": False,
    "LEFT MIDDLE": False,
    "LEFT RING": False,
    "LEFT PINKY": False}
    
    finger_tips = {"THUMB": 4,
                   "INDEX": 8,
                   "MIDDLE": 12,
                   "RING": 16,
                   "PINKY": 20}
 
    output_img = img.copy()
    rgbimg = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    result = hands.process(rgbimg)
    hand = {"RIGHT": False, "LEFT": False}
    counts = {"RIGHT": 0, "LEFT": 0}
    if result.multi_handedness:
        for hand_index, handLmk in enumerate(result.multi_handedness):
            hand_label = handLmk.classification[0].label.upper()
            hand[hand_label] = True
            hand_landmarks = result.multi_hand_landmarks[hand_index]
            wrist = (hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y)
            for key, val in finger_tips.items():
                finger = hand_label + " " + key
                tip = (hand_landmarks.landmark[val].x, hand_landmarks.landmark[val].y)
                if key == "THUMB":
                    knuckle = (hand_landmarks.landmark[13].x, hand_landmarks.landmark[13].y)
                    joint = (hand_landmarks.landmark[3].x, hand_landmarks.landmark[3].y)
                    finger_status[finger] = math.dist(tip, knuckle) > math.dist(joint, knuckle)
                else:
                    joint1 = (hand_landmarks.landmark[val - 1].x, hand_landmarks.landmark[val - 1].y)
                    joint2 = (hand_landmarks.landmark[val - 1].x, hand_landmarks.landmark[val - 1].y)
                    finger_status[finger] = math.dist(tip, wrist) > math.dist(joint, wrist)
#                 if math.dist(tip, wrist) > math.dist(joint, wrist) - delta:
#                     print("YAy", finger, finger_status[finger])
               
                if finger_status[finger]:
                    counts[hand_label] += 1


    if result.multi_hand_landmarks and draw:
        for hand_landmark in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(image = output_img, landmark_list = hand_landmark,
                                      connections = mp_hands.HAND_CONNECTIONS,
                                      landmark_drawing_spec = mp_draw.DrawingSpec(color=(0,0,255),
                                                                                  thickness=5, circle_radius=5),
                                      connection_drawing_spec=mp_draw.DrawingSpec(color = (255,255,255),
                                                                                     thickness = 10, circle_radius = 10))
    return output_img, result, counts, finger_status, hand
                

### FINGER Counter???

In [14]:
cap = cv.VideoCapture(0)
ptime = ctime = 0

while cap.isOpened():
    ret, img = cap.read()
    if not ctime:
        h, w, c = img.shape
    if ret:
        drawn_img, results, count, finger_status, hand = fingerCounter(img, hands, True)
        ctime = time.time()
        fps = 1/(ctime-ptime)
        ptime = ctime
        dist = -1
        cv.putText(drawn_img,f'FPS: {int(fps)}', (50, 70), font, fontScale,fontColor,thickness,lineType)
        # Labeling the landmarks also calculates if thumbs is highest y coordinate
        cv.putText(drawn_img,str(count['RIGHT'] + count['LEFT']), (4 * w // 5, h // 4), font, 5,(0,255,0),thickness,lineType)
        if hand['RIGHT']:
            if not count['RIGHT'] or (count['RIGHT'] == 1 and finger_status['RIGHT THUMB']):
                cv.putText(drawn_img,"ROCK", (8 * w // 11, h // 2), font, 5,(0,0,0),thickness,lineType)
            elif count['RIGHT'] == 5:
                cv.putText(drawn_img,"PAPER", (8 * w // 11 , h // 2), font, 5,(0,0,0),thickness,lineType)
            elif 2 <= count['RIGHT'] <= 3 and not (finger_status['RIGHT PINKY'] or finger_status['RIGHT RING']) and (finger_status['RIGHT INDEX'] and finger_status['RIGHT MIDDLE']):
                cv.putText(drawn_img,"SCISSORS", (6 * w // 11 , h // 2), font, 5,(0,0,0),thickness,lineType)
        if hand['LEFT']:
            if not count['LEFT'] or (count['LEFT'] == 1 and finger_status['LEFT THUMB']):
                cv.putText(drawn_img,"ROCK", (0 , h // 2), font, 5,(0,0,0),thickness,lineType)
            elif count['LEFT'] == 5:
                cv.putText(drawn_img,"PAPER", (0 , h // 2), font, 5,(0,0,0),thickness,lineType)
            elif 2 <= count['LEFT'] <= 3 and not (finger_status['LEFT PINKY'] or finger_status['LEFT RING']) and (finger_status['LEFT INDEX'] and finger_status['LEFT MIDDLE']):
                cv.putText(drawn_img,"SCISSORS", (0 * w // 11 , h // 2), font, 5,(0,0,0),thickness,lineType)
        if results.multi_hand_landmarks:
            for handLMS in results.multi_hand_landmarks:
                sx = int(handLMS.landmark[4].x * w)
                sy = int(handLMS.landmark[4].y * h)
                ex = int(handLMS.landmark[8].x * w)
                ey = int(handLMS.landmark[8].y * h)
                drawn_img = cv.line(drawn_img, (sx, sy), (ex, ey), (0,255,255), 2)
                dist = math.dist((sx, sy), (ex, ey))
        #cv.putText(drawn_img,"DIST APART "+ str(dist), (0,h//2), font, 5,(255,255,255),thickness,lineType)
        if dist!=-1: 
            if dist <= 50:
                target_volume = 0
            elif dist >= 500:
                target_volume = 100
            else:
                target_volume = int(((dist - 50)/450) * 100)
            osascript.osascript("set volume output volume {}".format(target_volume))
            drawn_img = cv.line(drawn_img, (sx, sy), (ex, ey), (0, 255, 0), thickness=5)
        cv.imshow("Webcam", drawn_img)
        
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv.destroyAllWindows()

KeyboardInterrupt: 

In [7]:
import torch

ModuleNotFoundError: No module named 'torch'